## **create**

In [ ]:

import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr

def lambda_handler(event, context):
    for k in range(9,5350,1):
        k=str(k)
        userid="imsi-20893"+str(k.zfill(10))
        ueID = userid
        dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
        table = dynamodb.Table('Subscribe_User')
        dynamodb_client = boto3.client('dynamodb')
        startSession = table.put_item(
            Item={
                'ue_ID': ueID,
                'Units': "4.8"
            }
        )
       
      
    output = " Create Successful!!" 
    response = {'statusCode': 200, 'body' : output}
    return response
    

##**get Subscribe_User**

In [ ]:
#get Subscribe_User 
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    ueID = event['ue_ID']
    dynamodb_resource = boto3.resource('dynamodb')
    suscribetable=dynamodb_resource.Table('Subscribe_User')
    GU_old = suscribetable.query(
        KeyConditionExpression=Key('ue_ID').eq(ueID)
    )
    for i in GU_old['Items']:
        GU_value=i['Units']
    output=GU_value
    response = {'statusCode': 200, 'body' : output}
    return response

## **get_dynamodb_ueid**

In [ ]:
#get_dynamodb_ueid
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    ueID = event['ue_ID']
    Time = event['datetime']
    dynamodb_resource = boto3.resource('dynamodb')
    #uetable = dynamodb_resource.Table(ueID)
    uetable = dynamodb_resource.Table('ADRF')
    # GU_cost = uetable.query(
    #     KeyConditionExpression=Key('datetime').eq(Time)
    # )
    GU_cost = uetable.get_item(Key={'ueid': ueID, 'datetime': Time})
     
    # for i in GU_cost['Items']:
    #     GU_cumsume=i['Internet']
    rea=GU_cost['Item']
    GU_cumsume=rea['Internet']
    
    output=GU_cumsume
    response = {'statusCode': 200, 'body' : output}
    return response
    


## **put_dynamodb**

In [ ]:
#put_dynamodb
import boto3, json, os, datetime as dt
from random import randrange
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    #body = json.loads(event['body'])
    ueID = event['ue_ID']
    value= event['value']
    dynamodb_resource = boto3.resource('dynamodb')
    
    suscribetable=dynamodb_resource.Table('Subscribe_User')
    GU_old = suscribetable.query(
        KeyConditionExpression=Key('ue_ID').eq(ueID)
    )
  

    suscribetable.update_item(
    Key={ 'ue_ID': ueID,},
    UpdateExpression="set Units=:u",
    ExpressionAttributeValues={
       ':u': value
    },
    )
    
    response = {'statusCode': 200, 'body' : "put ok"}
    return response
    


## **putGU**

In [ ]:

import json
import boto3
from boto3.dynamodb.conditions import Key, Attr
def lambda_handler(event, context):
    for k in range(1,256,1):
        k=str(k)
        userid="imsi-20893"+str(k.zfill(10))
        ueID = userid
        dynamodb = boto3.resource('dynamodb', region_name='us-east-1')
        table = dynamodb.Table('Subscribe_User')
        dynamodb_client = boto3.client('dynamodb')
        startSession = table.put_item(
            Item={
                'ue_ID': ueID,
                'GU': "4.88"
            }
        )
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }


## **simulate**

In [ ]:
from decimal import Decimal
import json
import urllib3
http = urllib3.PoolManager()
import numpy as np
import joblib
from google.colab import drive

class bcolors:
    OK = '\033[92m' #GREEN
    WARNING = '\033[93m' #YELLOW
    FAIL = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

def Nchf_ConvergedChargingFunction_create(ueID):
    #create UE table
    response=ueID+" Create Successful!!"
    return response

def getGuvalues(ueID):
    data = json.dumps({'ue_ID':ueID})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response

def getInternetvalues(ueID,time):
    data = json.dumps({'ue_ID':ueID,'datetime':time})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    response=response['body']
    return response

def putGU(ueID,value):
    data = json.dumps({'ue_ID':ueID,'value':value})
    URL=''
    response = http.request('POST',URL, body=data)
    response=response.data.decode()
    response=json.loads(response)
    return response

def Nchf_ConvergedChargingFunction_writingsession(ueID,time): 
    GU_old=float(getGuvalues(ueID))
    GU_consume=float(getInternetvalues(ueID,time))
    GU_new=GU_old-GU_consume
    GU_new=str(GU_new)
    putGU(ueID,GU_new)
    response=time+" "+bcolors.OK + " [CTF]" + bcolors.RESET+ueID+ " Collecting Charging Event."
    print(response)

def Nchf_ConvergedChargingFunction_update(ueID,time):
  data_GU=getGuvalues(ueID)
  data_internet=getInternetvalues(ueID,time)
  data_GU=float(data_GU)
  if data_GU<=0.0:
    predict_data=[data_internet]
    predict_data=np.array(predict_data)
    predict_data = np.reshape(predict_data, (predict_data.shape[0], 1, 1))
    GU_predict = modeltest.predict(predict_data)
    data_GU=abs(data_GU)
    GU_predict=float(GU_predict)
    data_GU=GU_predict+data_GU
    data_GU=str(data_GU)
    putGU(userid,data_GU)
    print(time+" "+bcolors.FAIL + " [CHF]" + bcolors.RESET+ueID+ " GU exhausted, Updating...")
  else:
    print(time+" "+bcolors.OK + " [CHF]" + bcolors.RESET+ueID+ " GU enough, no need to update.")


for i in range(1,8,1):
  i=str(i)
  for j in range(1,24,1):
    j=str(j)
    j=j.zfill(2)
    datetime="2013/11/"+i+" "+j+":00"
    for k in range(0,5350,1):
      k=str(k)
      userid="imsi-20893"+str(k.zfill(10))
      Nchf_ConvergedChargingFunction_writingsession(userid,datetime)
      Nchf_ConvergedChargingFunction_update(userid,datetime)